<a href="https://colab.research.google.com/github/leemordechai/FLAME/blob/master/FLAME_PAS_CoinGroups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://finds.org.uk/database/hoards/record/id/1788

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import savefig

%matplotlib inline

from google.colab import files
import pandas as pd
import datetime as dt
import io
import ast

import urllib3
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen, Request
import re
import pandas as pd
import datetime
import time

In [ ]:
_ = files.upload()
complete_finds = pd.read_csv('complete_finds.csv')
complete_finds.head(1)

Saving complete_finds.csv to complete_finds.csv


,Unnamed: 0,hoard_id,hoard_gis_id,unique_id,name,startDate,endDate,type_find,hoard?,excavation?,single?,num_coins,num_known_coins,num_unknown_coins,year_found,year_end_found,comments,lat,long,certainty,owner,created,imported
0,0,4.0,2083.0,IARCH-15228D,PAS: Newton Mills 4,383.0,388.0,EG_IARCH,hoard,NaN,NaN,255.0,255.0,0.0,1983.0,1983.0,"Robertson 2000, 358 no. 1474:""The hoard of 255...",51.3833,-2.418104,highest,PAS UK Finds,NaN,2021-04-15 18:44:33.679105


In [ ]:
import urllib.request, json 
with urllib.request.urlopen('https://finds.org.uk/database/hoards/record/id/' + str(2214) + '/format/json') as url:
    coin_find_data = json.loads(url.read().decode())
print(coin_find_data)

['record', {'id': '2214', 'hoardID': 'PASD69E8A82001698', 'uniqueID': 'PASD69E8A82001698', 'broadperiod': 'ROMAN', 'secwfstage': '3', 'treasure': None, 'recorderID': '00143623F1801460', 'created': '2015-01-13 11:57:26', 'createdBy': '528', 'updated': '2015-10-09 15:31:49', 'updatedBy': '528', 'institution': 'IARCH', 'knownas': 'Cunetio (well)', 'alsoknownas': None, 'period1': None, 'subPeriodFrom': None, 'period2': None, 'subPeriodTo': None, 'numdate1': None, 'numdate2': None, 'periodFrom': None, 'periodTo': None, 'lastrulerID': '133', 'reeceID': '19', 'terminalDateFrom': '367', 'terminalDateTo': '378', 'terminalreason': '1', 'lastRuler': 'House of Valentinian', 'lastReecePeriod': 'Valentinianic', 'date_range': '364-378', 'terminalReason': 'Date of latest coin', 'description': '<p>Robertson 2000, 336 no. 1369A:</p>\n\n<p>"During the clearing out of a late-Roman well at Cunetio (Mildenhall), about 100 Roman coins were found. They were identified as ranging from Tetricus II to Valentinia

In [ ]:
def get_coin_group_data(find_json, group_id):
  try:  
    coin_group_data = find_json[1][group_id]
  except:
    return False
 
  try:
    coin_group = dict()
    coin_group['denomination'] = coin_group_data['denomination']
  except:
    return -1
  
  coin_group['coin_group_id'] = coin_group_data['summaryID']
  try:
    coin_group['num_coins'] = int(coin_group_data['quantity'])
  except:
    coin_group['num_coins'] = -1
  coin_group['mint'] = coin_group_data['mint']
  coin_group['start_year'] = coin_group_data['numdate1']
  coin_group['end_year'] = coin_group_data['numdate2']
  coin_group['ruler'] = coin_group_data['ruler']
  coin_group['period'] = coin_group_data['broadperiod']
  coin_group['created'] = datetime.datetime.now()
  coin_group['imported'] = datetime.datetime.now()
  return coin_group

Run this to download the Coin Groups


In [ ]:
coin_group_df = pd.DataFrame(columns=['hoard_id', 'hoard_id_long', 'coin_group_id', 'period', 'start_year', 'end_year', 'revised_start', 'revised_end', 'ruler', 'revised_ruler', 'denomination', 'num_coins', 'mint','imported', 'created','updated'])
#list_of_hoard_ids = [3103, 498]
list_of_hoard_ids = [int(i) for i in complete_finds['hoard_id']] # as taken from the complete_finds csv file, takes 10min to run
import urllib.request, json 
entries_with_errors = []

for hoard in list_of_hoard_ids:
  try:
    with urllib.request.urlopen('https://finds.org.uk/database/hoards/record/id/' + str(hoard) + '/format/json') as url:
      coin_find_data = json.loads(url.read().decode())
  except:
    print('Error in hoard_id: ' + str(hoard))
  coin_find_id = coin_find_data[1]['id']
  coin_find_description = coin_find_data[1]['description']
  coin_find_notes = coin_find_data[1]['notes']
  coin_find_hoard_id = ''

  if (hoard % 10) == 0: print(hoard)
  k = 0
  while k >= 0:
    try:
      coin_find_group = coin_find_data[1][str(k)]
    except:
      print('problem in find ' + str(hoard))
      entries_with_errors = entries_with_errors + [hoard]
      break
    try:
      coin_find_hoard_id = coin_find_group['recordID']
      break
    except:
      k = k+1
  coin_find_time_start = coin_find_data[1]['datefound1']
  coin_find_time_end = coin_find_data[1]['datefound2']

  i = k
  while i >= 0:
    ans = get_coin_group_data(coin_find_data, str(i))
    i = i+1
    if ans == False: break
    elif ans == -1: break
    else:
      ans['hoard_id'] = coin_find_id
      ans['hoard_id_long'] = coin_find_hoard_id
      coin_group_df = coin_group_df.append(ans, ignore_index=True)

coin_group_df['start_year'] = coin_group_df['start_year'].apply(pd.to_numeric, errors='ignore')
coin_group_df['end_year'] = coin_group_df['end_year'].apply(pd.to_numeric, errors='ignore')
coin_group_df
# errors in hoard_id 479, 1984


10
40
60
140
170
problem in find 176
180
220
problem in find 235
240
250
problem in find 274
310
problem in find 310
360
370
problem in find 399
problem in find 407
450
problem in find 450
Error in hoard_id: 479
510
problem in find 516
520
problem in find 543
580
problem in find 598
600
problem in find 600
problem in find 639
640
660
problem in find 676
problem in find 677
690
problem in find 694
700
710
720
problem in find 722
problem in find 773
problem in find 785
790
800
810
problem in find 810
820
840
problem in find 867
870
890
920
problem in find 925
problem in find 938
960
problem in find 962
980
990
1010
1030
problem in find 1032
1060
1150
1160
problem in find 1168
1180
problem in find 1254
1260
1280
1330
1340
1350
1360
problem in find 1362
1390
problem in find 1411
problem in find 1442
problem in find 1446
1460
1470
problem in find 1518
problem in find 1537
problem in find 1538
1540
problem in find 1540
1620
1650
1700
1730
1740
1760
1770
problem in find 1770
1800
1860
problem

,hoard_id,hoard_id_long,coin_group_id,period,start_year,end_year,revised_start,revised_end,ruler,revised_ruler,denomination,num_coins,mint,imported,created,updated
0,4,IARCH-15228D,257,ROMAN,NaN,350.0,NaN,NaN,Constans,NaN,Siliqua,1,Trier,2021-04-16 16:54:57.623244,2021-04-16 16:54:57.623230,NaN
1,4,IARCH-15228D,327,ROMAN,355.0,364.0,NaN,NaN,Constantius II,NaN,Siliqua,3,Arelatum,2021-04-16 16:54:57.631005,2021-04-16 16:54:57.630995,NaN
2,4,IARCH-15228D,328,ROMAN,355.0,364.0,NaN,NaN,Constantius II,NaN,Siliqua,1,Nicomedia,2021-04-16 16:54:57.640365,2021-04-16 16:54:57.640354,NaN
3,4,IARCH-15228D,329,ROMAN,361.0,363.0,NaN,NaN,Julian,NaN,Siliqua,4,Trier,2021-04-16 16:54:57.649578,2021-04-16 16:54:57.649567,NaN
4,4,IARCH-15228D,330,ROMAN,361.0,363.0,NaN,NaN,Julian,NaN,Siliqua,1,Lugdunum,2021-04-16 16:54:57.656869,2021-04-16 16:54:57.656858,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10267,2286,IARCH-CC3BAF,25138,ROMAN,NaN,NaN,NaN,NaN,Galerius,NaN,Nummus (AE 1 - AE 4),-1,None,2021-04-16 17:08:25.225174,2021-04-16 17:08:25.225162,NaN
10268,2286,IARCH-CC3BAF,25139,ROMAN,NaN,NaN,NaN,NaN,Maximinus Daia,NaN,Nummus (AE 1 - AE 4),-1,None,2021-04-16 17:08:25.243683,2021-04-16 17:08:25.243671,NaN
10269,269,IARCH-E1B4C6,26225,ROMAN,307.0,313.0,NaN,NaN,House of Constantine,NaN,Nummus (AE 1 - AE 4),4,Trier,2021-04-16 17:08:25.743345,2021-04-16 17:08:25.743331,NaN
10270,269,IARCH-E1B4C6,26226,ROMAN,315.0,316.0,NaN,NaN,House of Constantine,NaN,Nummus (AE 1 - AE 4),1,Londinium,2021-04-16 17:08:25.760714,2021-04-16 17:08:25.760702,NaN


In [ ]:
print(entries_with_errors) # [176, 235, 274, 310, 399, 407, 450, 516, 543, 598, 600, 639, 676, 677, 694, 722, 773, 785, 810, 867, 925, 938, 962, 1032, 1168, 1254, 1362, 1411, 1442, 1446, 1518, 1537, 1538, 1540, 1770, 1973, 2080, 2110, 2173, 2188, 2196, 2198, 2220, 2241, 2245, 2313, 2329, 2374, 2927, 2937, 3021, 3032, 3117, 3126, 3190, 3200, 3213, 3224, 3232, 3253, 3256, 632, 1639, 2047, 613, 3168, 3139, 289, 2527, 509, 3183, 3196, 1559, 3205, 385, 3193, 3194, 1290, 483, 849, 3254, 3259, 3030, 3240, 712, 3246, 692, 2891, 3191, 3245, 3258, 3048, 3201, 3249, 44, 1399, 3175, 826, 3195, 3212, 3239, 3056, 1430, 2759, 1191, 2058, 1743, 2287, 1921, 1366, 1102, 1790, 3171, 1825, 3111, 3138, 871, 1463, 2409, 930, 861, 1775, 2031, 1791, 789, 1220, 2377, 3152, 3100, 3046, 596, 2975, 2344, 2089, 3132, 3108, 427, 1270, 287, 460, 586, 2388, 58, 280, 2373, 350, 390, 940, 1200, 359, 318, 368, 1223, 1472, 349, 3053, 3155, 3157, 1767, 1251, 2275, 299, 1487, 458, 2277, 2219, 2338, 597, 1511, 1094, 721, 3006, 1132, 1236, 334, 1192, 1031, 1480, 1850, 2301, 3124, 331, 991, 3051, 295, 337, 511, 2291, 1176, 689, 2208, 3004, 1786, 614, 626, 1135, 2238, 1772, 2556, 3129, 1269, 1372, 272, 296, 2181, 704, 226, 1325, 467, 1516, 1877, 575, 2521, 2392, 1104, 3031, 926, 3160, 1979, 1971, 615, 725, 2255, 758, 1076, 2478, 1591, 3011, 3033, 1263, 2276, 2228, 2335, 734, 143, 1108, 3192, 2304, 1698, 1678, 581, 2115, 587, 2000, 2315, 666, 1593, 2150, 2352, 2415, 696, 1194, 2229, 1024, 2339, 1170, 791, 278, 1500, 1894, 1830, 1587, 466, 2479, 413, 705, 2955, 825, 3133, 1227, 1467, 606, 1862, 410, 1394, 1758, 2048, 747, 759, 300, 2215, 185, 1729, 473, 1691, 505, 1679, 852, 1090, 388, 1195, 2956, 1714, 1629, 1983, 455, 2321, 1967, 1489, 1696, 1945, 1660, 2267, 1724, 2440, 562, 1823, 1917, 1653, 1092, 3141, 2183, 227, 1248, 1929, 3180, 727, 2550, 2018, 1898, 3025, 419, 311, 434, 2308, 1380, 2416, 3148, 706, 583, 642, 741, 1936, 2385, 493, 2293, 3145, 1130, 238, 3055, 3034, 73, 736, 2036, 1371, 1701, 519, 462, 783, 253, 2179, 3154, 3107, 1997, 1763, 1512, 1995, 2281, 494, 320, 2378, 1597, 812, 2547, 436, 1590, 3142, 1086, 3114, 2350, 796, 377, 59, 1510, 361, 1205, 2083, 1153, 1381]
print(len(entries_with_errors)) # 379 entries

[176, 235, 274, 310, 399, 407, 450, 516, 543, 598, 600, 639, 676, 677, 694, 722, 773, 785, 810, 867, 925, 938, 962, 1032, 1168, 1254, 1362, 1411, 1442, 1446, 1518, 1537, 1538, 1540, 1770, 1973, 2080, 2110, 2173, 2188, 2196, 2198, 2220, 2241, 2245, 2313, 2329, 2374, 2927, 2937, 3021, 3032, 3117, 3126, 3190, 3200, 3213, 3224, 3232, 3253, 3256, 632, 1639, 2047, 613, 3168, 3139, 289, 2527, 509, 3183, 3196, 1559, 3205, 385, 3193, 3194, 1290, 483, 849, 3254, 3259, 3030, 3240, 712, 3246, 692, 2891, 3191, 3245, 3258, 3048, 3201, 3249, 44, 1399, 3175, 826, 3195, 3212, 3239, 3056, 1430, 2759, 1191, 2058, 1743, 2287, 1921, 1366, 1102, 1790, 3171, 1825, 3111, 3138, 871, 1463, 2409, 930, 861, 1775, 2031, 1791, 789, 1220, 2377, 3152, 3100, 3046, 596, 2975, 2344, 2089, 3132, 3108, 427, 1270, 287, 460, 586, 2388, 58, 280, 2373, 350, 390, 940, 1200, 359, 318, 368, 1223, 1472, 349, 3053, 3155, 3157, 1767, 1251, 2275, 299, 1487, 458, 2277, 2219, 2338, 597, 1511, 1094, 721, 3006, 1132, 1236, 334, 1192, 10

Can upload a csv of all the coin groups downloaded from PAS instead of running the script

In [ ]:
_ = files.upload()
coin_group_df = pd.read_csv('coin_groups_online_version (2).csv')

Saving coin_groups_online_version (2).csv to coin_groups_online_version (2).csv


In [ ]:
#@title Initial filtering of denominations, rulers, mints and specific entries
def initial_filtering(coin_groups_raw):		# filters out irrelevant denominations, rulers, mints, and entries
	have_bad_dates = coin_groups_raw[coin_groups_raw.end_year < 325]	# works

	list_of_bad = have_bad_dates.index.tolist()	# prepare for filtering out
	coin_groups_filtered = coin_groups_raw.drop(list_of_bad)	# drop all irrelevant rows

	irrelevant_denominations = ['Radiate (antoninianus)', 'Sestertius', 'Denarius (Empire)', 
								'Denarius (Roman Republic)', 'Dupondius', 'Quinarius',
								'As (Roman Republic)', 'Quadrans (Roman Republic)', 'Quadrans',
								'Sestertius (Roman Republic)', 'Dupondius or as', 'Double sestertius',
								'Sestertius, dupondius or as', 'Q radiate', 'As', 'Semis', 'Stater (gold)',
								'Aureus (Republic/Empire)', 'Denarius', 'Stater (silver)', 
								'Quarter stater (gold)', 'Quarter stater (silver)', 'Half stater', 'Denarius',
								'As', 'Denarius (Empire)', 'Unit (copper alloy)', 'Drachm', 'Tetradrachm',
								'Potin (Cast bronze)']
	for irr_den in irrelevant_denominations:
		print('{} of "{}" removed.'.format(len(coin_groups_filtered[coin_groups_filtered.denomination == irr_den]), irr_den))
		coin_groups_filtered = coin_groups_filtered[coin_groups_filtered.denomination != irr_den]

	print('Overall, {} coin groups remain in the database and ready for import'.format(len(coin_groups_filtered)))
	print('This is the list of coin denominations that remains in the database: {}'.format(coin_groups_filtered.denomination.unique()))
	print()

	# Removes all irrelevant rulers from the data frame
	irrelevant_rulers = ["Julio-Claudian (uncertain)", "Caligula", "Claudius", "Vespasian", "Marcus Aurelius (as Augustus)",
		"Lucilla", "Antonine Empress, uncertain, 138-185", "Uncertain - 1st/2nd Century AD", 'Licinius I', 
		'Diocletian', 'Constantius I', 'Maximian I', 'Galeria Valeria', 'Nero', 'Galba', 'Otho', 'Vitellius',
		'Titus', 'Trajan', 'Hadrian', 'Sabina', 'Nerva', 'Domitian', 'Antoninus Pius', 'Aelius Caesar', 'Maxentius',
		'Marciana', 'Maximinus Daia', 'Tetrarchic Ruler (uncertain issuer)', 'Faustina II', 'Caracalla', 'Philip I',
		'Philip II', 'Gordian III', 'Severus Alexander', 'Elagabalus', 'Uncertain - 1st-mid 3rd century', 'Probus',
		'Victorinus', 'Galerius', 'Divus Claudius (Official)', 'Aurelian', 'Gallienus (sole reign)', 
		'Salonina (sole reign of Gallienus)', 'Radiate, Uncertain Ruler 260-296', 'Balbinus', 'Licinius II', 
		'Radiate (Barbarous)', 'Balbinus', 'Julia Maesa', 'Severus', 'Carausius', 'Augustus', 'Hostilian',
		'Tiberius', 'Postumus', 'Maximinus', 'Juba I', 'Tranquillina', 'Trebonius Gallus', 'Trajan Decius',
		'Trajan', 'Julia Domna', 'ECEN/ECE', 'AESV', 'Uninscribed', 'Cunobelin', 'Gelon I And Hieron I',
		'Valerian I', 'Epaticcus', 'Gallienus', 'Commodus', 'Claudius II', 'Carinus', 
		'VOLISIOS DUMNOVELLAUNOS', 'VOLISIOS CARTIVELLAVNOS', 'SAENV', 'IISVPRASV', 'DUMNOCO TIGIR SENO',
		'Gallienus', 'Otacilia', 'Tincomarus', 'Verica', 'Julia Mamaea', 'CARA[TACUS]',
		'EISV', 'AVN COST', 'ANTED (Iceni)', 'CANI DVRO', 'ANTED (Dobunni)', 'Eppillus', 
		'VOLISIOS DUMNOCOVEROS', 'Salonina', 'VEP CORF', 'DVMNOCO TIGIR SENO', 'BODVOC', 
		'SVB ESVPRASTO ESICO FECIT']
	for irr_ruler in irrelevant_rulers:
		coin_groups_filtered = coin_groups_filtered[coin_groups_filtered.ruler != irr_ruler]

	irrelevant_mints = ['Ostia']
	for irr_mint in irrelevant_mints:
		coin_groups_filtered = coin_groups_filtered[coin_groups_filtered.mint != irr_mint]

	# manual removal of entries
	irrelevant_entries = [] 
							# nothing/very little known about these coins and error in ruler attribution
	for irr_entry in irrelevant_entries:
		coin_groups_filtered = coin_groups_filtered[coin_groups_filtered.coin_group_id != irr_entry]

	return coin_groups_filtered

In [ ]:
_ = files.upload()
flame_ruler_list = pd.read_excel('Rulers.xlsx')

Saving Rulers.xlsx to Rulers.xlsx


In [ ]:
#@title Second filtering of denominations, rulers, mints and specific entries
def coin_group_cleaning(coin_groups, flame_rulers):
	#flame_denominations = pd.read_excel('Denominations.xlsx')
	#flame_mints = pd.read_excel('Mints.xlsx')
	#flame_rulers = pd.read_excel('Rulers.xlsx')

	# setting conversions
	ruler_list = {"House of Constantine":(307, 363), "House of Valentinian":(364,378), "House of Theodosius":(378, 408),
		"Magnentius":(350,353), "Uncertain (AD 260 - 402)":(260, 402), 
		'Uncertain - 4th century':(300, 399), 'Magnentius or Decentius': (350, 353),
		'Flavius Victor':(384, 388), 'Dalmatius':(335, 337),
		'Constantinopolis':(330, 341),	# based on the existing entries in the database
		'Procopius': (365, 366),
		'Constans': (330, 350),
		'Fausta': (324, 330),			# based on the existing entries in the database
		'Urbs Roma': (330, 341),		# based on the existing entries in the database
		'Crispus': (311, 330),			# based on the existing entries in the database
		'Theodora': (337, 341),			# based on the existing entries in the database
		'Magnus Maximus or Flavius Victor': (384, 388),
		'Helena': (324, 341),
		'Constantius Gallus': (351, 355), 
		'Decentius': (350, 353),
		'Honorius (emperor)': (395, 423),
		'Nummus, uncertain ruler, c. 330-402': (330, 402)
		}
	for i in range(len(flame_rulers)):
		ruler_list[flame_rulers.at[i, 'RulerName']] = (flame_rulers['RulerStartYear'].iloc[i],flame_rulers['RulerEndYear'].iloc[i])

	denomination_dates = {"Nummus (AE 1 - AE 4)": (302, 402),	# based on existing entries
						'AE 1-4 (UK find)': (302, 402), # based on existing entries
						"Radiate or nummus": (260, 402),		# based on existing entries (/w corrections)
						"Siliqua": (360, 402),					# based on existing entries
            "siliqua": (360, 402),        # based on existing entries
            "radiate or nummus (UK find)": (260, 402), # based on existing entries (/w corrections)
						"Uncertain (copper alloy)":(-100, 410),	
						"Uncertain (silver)": (-100, 410),		
						"Unspecified ruler (contemporary copy)":(-100, 410), 
            "solidus": (300, 500), # based on existing entries
            "miliarensis": (306, 395), # based on existing entries
            "tremissis": (383, 600), # based on existing entries & rough cut off
						"Nummus, uncertain ruler, c. 330-402": (300, 402),
            "1/2 siliqua": (378, 408)
						}
	# order: 	mint name in UK database:mint name in FLAME
	mint_conversion = {"Trier": "Colonia Augusta Treverorum", "Lyon": "Lugdunensium", "Lugdunum":"Lugdunensium",  
					"Rome": "Roma",	"Thessalonica": "Thessalonika", "Siscia": "Siscia", "Aquileia": "Aquileia", 
					"Milan": "Mediolanum", 'Amiens (Ambianum)': 'Ambianum', 'Nicomedia':'Nikomedia', 
					"Heraclea":"Heraclea", "London":"Londinium", "Antioch":"Antioch", "Arles": "Arelato",
					"Pavia":"Ticinum", "Cyzicus":"Kyzikos", 'Sirmium':'Sirmium', 
					'Constantinople':'Constantinople', "Ravenna": "Ravenna", "Alexandria": "Alexandria",
					'Unattributed': 'Unknown', 'Eastern mint': 'Unknown (East Roman)', 
					'Gallic mint': 'Unknown (Gaul)', 'Arles or Lyons ': 'Arelato or Lugdunum',
					'Italian mint': 'Unknown (Italy)', 'Alexandria (Egypt)': 'Alexandria',
          'Antioch (Antakya, Turkey)':'Antioch', 'Arelatum':'Arelatum',
          'Mediolanum':'Mediolanum', 'Ambianum':'Ambianum', 'Londinium':'Londinium',
          'Alexandria, Egypt':'Alexandria', 'Ticinum':'Ticinum', 'Perinthus/Heraclea':'Heraclea'}
	# order: 	denomination in UK database:denomination name in FLAME
	denomination_conversion = {'Nummus (AE 1 - AE 4)':"AE 1-4 (UK find)",	# new denomination (bronze)
							'Miliarensis':"miliarensis",
							'Siliqua': "siliqua",
							'Radiate or nummus':"radiate or nummus (UK find)", # new denomination (bronze)
							'Solidus':"solidus",
							'Half unit (silver)':"half unit", 	# new denomination (silver)
							'Unit (silver)': "unit",			# new denomination (silver)
							'Half-siliqua':"1/2 siliqua",
							'Tremissis':"tremissis",
							'Uncertain':"uncertain",
							'Uncertain (gold)':"uncertain (gold)",		# new denomination (gold)
							'Uncertain (silver)':"uncertain (silver)",	# new denomination (silver)
							'Uncertain (copper alloy)':"unidentified bronze coins"
							}
	ruler_conversion = {'Honorius (emperor)': 'Honorius',			# add at least some of this to FLAME database
						'Nummus, uncertain ruler, c. 330-402': 'Unknown',
						'Uncertain': 'Unknown',
						'Uncertain - 4th century': 'Unknown',
						'Uninscribed': 'Unknown',
						'Unattributed': 'Unknown',
						'Dalmatius': 'Constantine I',
						'Magnentius or Decentius': 'Magnentius or Decentius',
						'Flavius Victor': 'Flavius Victor', 
						'Dalmatius': 'Dalmatius',
						'Constantinopolis': 'Constantinopolis',	
						'Procopius': 'Procopius',
						'Constans': 'Constans I',
						'Fausta': 'Fausta',			
						'Urbs Roma': 'Roma',		
						'Crispus': 'Crispus',			
						'Theodora': 'Theodora (4th century)',
						'Magnus Maximus or Flavius Victor': 'Magnus Maximus or Flavius Victor',
						'Helena': 'Helena',
						'Constantius Gallus': 'Constantius Gallus', 
						'Decentius': 'Decentius',
            '':'Unknown'
						}

	coin_groups['revised_start'] = coin_groups['start_year']
	coin_groups['revised_end'] = coin_groups['end_year']
	coin_groups['revised_ruler'] = coin_groups['ruler']
	coin_groups['revised_ruler'] = coin_groups['revised_ruler'].fillna('Unknown')
	rulers_to_resolve = set()
	
	for i in coin_groups.index:
		# this updates revised_ruler based on the ruler_conversion dictionary. This is for display (not data extraction)
		ruler_to_check = coin_groups.at[i,'ruler']
		if ruler_to_check in ruler_conversion:
			coin_groups.at[i, 'revised_ruler'] = ruler_conversion[coin_groups.at[i,'ruler']]
		#print('ERROR:' + ruler_to_check)

    		# this section standardizes the mint names to those in FLAME's database
		try:
			if coin_groups.at[i, 'mint'] == coin_groups.at[i, 'mint']:
				try:
					coin_groups.at[i, 'mint'] = mint_conversion[coin_groups.at[i, 'mint']]
				except:
					print("Error: Unknown mint: {}".format(coin_groups.at[i, 'mint']))	# this should print nothing if working as intended
		except:
			continue

		# this section standardizes the denomination names to those in the FLAME database
		try:
			if coin_groups.at[i, 'denomination'] == coin_groups.at[i, 'denomination']:
				try:
					coin_groups.at[i, 'denomination'] = denomination_conversion[coin_groups.at[i, 'denomination']]
				except:
					print("Error: Unknown denomination: {}".format(coin_groups.at[i, 'denomination']))
		except:
			continue

		# this section fills in the dates based on ruler (or denomination) if they are missing
		if (coin_groups.at[i, 'revised_start'] != coin_groups.at[i, 'revised_start']):
			temp_start = -1
			if (coin_groups.at[i, 'revised_ruler'] != 'Unknown') and (coin_groups.at[i, 'revised_ruler'] != 'Unspecified ruler (contemporary copy)'):
				try:
					temp_start = ruler_list[coin_groups.at[i, 'ruler']][0]
				except:
					if coin_groups.at[i, 'ruler'] not in ruler_conversion:
						#print("Error: Unknown ruler: {}".format(coin_groups.at[i, 'ruler']))
						rulers_to_resolve.add(coin_groups.at[i, 'ruler'])
			else: #if coin_groups.at[i, 'revised_ruler'] == 'Unknown':
				temp_start = year_limit(denomination_dates, coin_groups.at[i, 'denomination'], "start")
				#print(i, coin_groups.at[i, 'revised_ruler'], coin_groups.at[i, 'denomination'])
			coin_groups.at[i, 'revised_start'] = temp_start

		if (coin_groups.at[i, 'revised_end'] != coin_groups.at[i, 'revised_end']):
			temp_end = -1
			if (coin_groups.at[i, 'revised_ruler'] != 'Unknown') and (coin_groups.at[i, 'revised_ruler'] != 'Unspecified ruler (contemporary copy)'):
				try:
					temp_end = ruler_list[coin_groups.at[i, 'ruler']][1]
				except:
					if coin_groups.at[i, 'ruler'] not in ruler_conversion:
						#print("Error: Unknown ruler: {}".format(coin_groups.at[i, 'ruler']))
						rulers_to_resolve.add(coin_groups.at[i, 'ruler'])
			else: #if coin_groups.at[i, 'ruler'] == 'Unspecified ruler (contemporary copy)':
				temp_end = year_limit(denomination_dates, coin_groups.at[i, 'denomination'], "end")
			coin_groups.at[i, 'revised_end'] = temp_end

	if len(rulers_to_resolve) > 1:	# the only entry here should be nan, otherwise print it all
		print("The unknown rulers are: {}".format(rulers_to_resolve))
  
	coin_groups['mint'] = coin_groups['mint'].fillna('Unknown')
	return coin_groups


In [ ]:
#@title Year limit - a minor function to help with cleaning Coin Groups
def year_limit(denom_list, denom, time):	# checks for relevant denominations
	if denom in denom_list:
		if time == "start": return denom_list[denom][0]
		if time == "end": return denom_list[denom][1]
	else: 
		print(denom)
		return -1

In [ ]:
coin_groups_first_filter = initial_filtering(coin_group_df)

663 of "Radiate (antoninianus)" removed.
46 of "Sestertius" removed.
55 of "Denarius (Empire)" removed.
3 of "Denarius (Roman Republic)" removed.
14 of "Dupondius" removed.
0 of "Quinarius" removed.
0 of "As (Roman Republic)" removed.
0 of "Quadrans (Roman Republic)" removed.
0 of "Quadrans" removed.
0 of "Sestertius (Roman Republic)" removed.
22 of "Dupondius or as" removed.
0 of "Double sestertius" removed.
31 of "Sestertius, dupondius or as" removed.
2 of "Q radiate" removed.
24 of "As" removed.
0 of "Semis" removed.
0 of "Stater (gold)" removed.
4 of "Aureus (Republic/Empire)" removed.
0 of "Denarius" removed.
2 of "Stater (silver)" removed.
0 of "Quarter stater (gold)" removed.
0 of "Quarter stater (silver)" removed.
0 of "Half stater" removed.
0 of "Denarius" removed.
0 of "As" removed.
0 of "Denarius (Empire)" removed.
0 of "Unit (copper alloy)" removed.
0 of "Drachm" removed.
0 of "Tetradrachm" removed.
0 of "Potin (Cast bronze)" removed.
Overall, 7658 coin groups remain in the

In [ ]:
coin_groups_second_filter = coin_group_cleaning(coin_groups_first_filter, flame_ruler_list)

Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unknown mint: None
Error: Unkno

In [ ]:
coin_groups_second_filter[coin_groups_second_filter['revised_start'].isna()] #0
coin_groups_second_filter[coin_groups_second_filter['revised_end'].isna()] #0, mostly overlap with the previous search
coin_groups_second_filter[coin_groups_second_filter['denomination'].isna()] #5
coin_groups_second_filter[coin_groups_second_filter['mint'].isna()] #0
coin_groups_second_filter[pd.to_numeric(coin_groups_second_filter['num_coins'], errors='coerce').isnull()] #0
coin_groups_second_filter[coin_groups_second_filter['revised_ruler'] == 'Unknown'] #0
coin_groups_second_filter[coin_groups_second_filter['revised_end'] == -1] #0


,hoard_id,hoard_id_long,coin_group_id,period,start_year,end_year,revised_start,revised_end,ruler,revised_ruler,denomination,num_coins,mint,imported,created,updated
356,114,IARCH-9355A6,2705,ROMAN,NaN,NaN,-1.0,-1.0,Uncertain,Unknown,uncertain,6,Unknown,2021-04-16 16:55:14.101354,2021-04-16 16:55:14.101342,NaN
1045,383,IARCH-1F511E,17205,ROMAN,NaN,NaN,-1.0,-1.0,Uncertain,Unknown,uncertain,1,Unknown,2021-04-16 16:55:51.241276,2021-04-16 16:55:51.241266,NaN
1299,492,IARCH-E36454,20583,ROMAN,NaN,NaN,-1.0,-1.0,Uncertain,Unknown,unidentified bronze coins,65,Unknown,2021-04-16 16:56:04.581263,2021-04-16 16:56:04.581253,NaN
1848,662,IARCH-BB720C,22092,ROMAN,NaN,NaN,-1.0,-1.0,Unspecified ruler (contemporary copy),Unspecified ruler (contemporary copy),unidentified bronze coins,-1,Unknown,2021-04-16 16:56:37.215384,2021-04-16 16:56:37.215372,NaN
2574,890,IARCH-9A209C,18305,ROMAN,NaN,NaN,-1.0,-1.0,Uncertain,Unknown,uncertain,8,Unknown,2021-04-16 16:57:14.264422,2021-04-16 16:57:14.264410,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9503,133,IARCH-FEF06A,1467,ROMAN,NaN,NaN,-1.0,-1.0,Uncertain,Unknown,uncertain,0,Unknown,2021-04-16 17:06:39.813812,2021-04-16 17:06:39.813801,NaN
9538,1042,IARCH-B59848,18391,UNKNOWN,NaN,NaN,-1.0,-1.0,Uncertain,Unknown,uncertain (silver),300,Unknown,2021-04-16 17:07:04.694976,2021-04-16 17:07:04.694965,NaN
9605,63,IARCH-F8660C,16341,UNKNOWN,NaN,NaN,-1.0,-1.0,Uncertain,Unknown,uncertain (gold),1,Unknown,2021-04-16 17:07:34.851581,2021-04-16 17:07:34.851569,NaN
9777,50,IARCH-23D743,14857,ROMAN,NaN,NaN,-1.0,-1.0,Uncertain,Unknown,uncertain,1,Unknown,2021-04-16 17:07:43.984506,2021-04-16 17:07:43.984493,NaN


In [ ]:
#download coins
#coin_group_df.to_csv('coin_groups_online_version.csv')
#files.download('coin_groups_online_version.csv')

coin_groups_second_filter.to_csv('coin_groups_online_filtered.csv')
files.download('coin_groups_online_filtered.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#coin_group_df[coin_group_df['num_coins'] >1000]
coin_groups_second_filter[coin_groups_second_filter['revised_start'].isna()]['denomination'].value_counts()

Series([], Name: denomination, dtype: int64)

In [ ]:
coin_groups_second_filter

,hoard_id,hoard_id_long,coin_group_id,period,start_year,end_year,revised_start,revised_end,ruler,revised_ruler,denomination,num_coins,mint,imported,created,updated
0,4,IARCH-15228D,257,ROMAN,NaN,350.0,330.0,350.0,Constans,Constans I,siliqua,1,Colonia Augusta Treverorum,2021-04-16 16:54:57.623244,2021-04-16 16:54:57.623230,NaN
1,4,IARCH-15228D,327,ROMAN,355.0,364.0,355.0,364.0,Constantius II,Constantius II,siliqua,3,Arelatum,2021-04-16 16:54:57.631005,2021-04-16 16:54:57.630995,NaN
2,4,IARCH-15228D,328,ROMAN,355.0,364.0,355.0,364.0,Constantius II,Constantius II,siliqua,1,Nikomedia,2021-04-16 16:54:57.640365,2021-04-16 16:54:57.640354,NaN
3,4,IARCH-15228D,329,ROMAN,361.0,363.0,361.0,363.0,Julian,Julian,siliqua,4,Colonia Augusta Treverorum,2021-04-16 16:54:57.649578,2021-04-16 16:54:57.649567,NaN
4,4,IARCH-15228D,330,ROMAN,361.0,363.0,361.0,363.0,Julian,Julian,siliqua,1,Lugdunensium,2021-04-16 16:54:57.656869,2021-04-16 16:54:57.656858,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10236,273,IARCH-4296C5,21199,ROMAN,NaN,NaN,260.0,402.0,Uncertain (AD 260 - 402),Uncertain (AD 260 - 402),radiate or nummus (UK find),5,Unknown,2021-04-16 17:08:20.363705,2021-04-16 17:08:20.363692,NaN
10239,30,IARCH-4DAD73,16310,ROMAN,NaN,NaN,306.0,337.0,Constantine I,Constantine I,AE 1-4 (UK find),-1,Unknown,2021-04-16 17:08:20.880068,2021-04-16 17:08:20.880057,NaN
10240,30,IARCH-4DAD73,16311,ROMAN,NaN,NaN,307.0,363.0,House of Constantine,House of Constantine,AE 1-4 (UK find),-1,Unknown,2021-04-16 17:08:20.897267,2021-04-16 17:08:20.897255,NaN
10257,2053,IARCH-8EEE13,22928,ROMAN,NaN,NaN,306.0,337.0,Constantine I,Constantine I,AE 1-4 (UK find),-1,Unknown,2021-04-16 17:08:24.059899,2021-04-16 17:08:24.059885,NaN


Take another look at these finds (among others): 91